- data downloaded from: http://hmp2-data.stanford.edu/index.php#
- reference study design: https://www.cell.com/cell-host-microbe/fulltext/S1931-3128(14)00306-0?_returnURL=http%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS1931312814003060%3Fshowall%3Dtrue
- shannon diversity: protein (take exponential to ensure non-negative values, than calculate shannon diversity)

In [126]:
library(dplyr)
library(tidyverse)
library(vegan)  # for shannon calculation: samples/subjects as rows, and omics as columns
library(ggplot2)
library(gridExtra) # ref: arrange multipe plots + labels: https://cran.r-project.org/web/packages/egg/vignettes/Ecosystem.html

### metadata information

In [38]:
meta = read.csv('../application_2_T2D/data/metadata.txt', sep='\t')

In [39]:
dim(meta)

[1] 1416    8

In [40]:
head(meta)

VisitID,SubjectID,CollectionDate,Event,Event_Note1,Event_Note2,Event_Note3,SubStudy
ZIS22OE-02,ZIS22OE,132,Healthy,,,,HMP
ZJBOZ2E11-E11,ZJBOZ2X,1481,Exercise,Baseline,,,Exercise
ZJBOZ2E12-E12,ZJBOZ2X,1481,Exercise,2 min,,,Exercise
ZJBOZ2E13-E13,ZJBOZ2X,1481,Exercise,15 min,,,Exercise
ZJBOZ2E14-E14,ZJBOZ2X,1481,Exercise,30 min,,,Exercise
ZJBOZ2E15-E15,ZJBOZ2X,1481,Exercise,1 hour,,,Exercise


In [41]:
hmp = meta[meta$SubStudy %in% 'HMP', ]
dim(hmp)

[1] 1092    8

In [42]:
head(table(hmp$CollectionDate))


-1068 -1064 -1047  -960  -952  -883 
    1     1     1     1     1     1 

In [43]:
tail(table(hmp$CollectionDate))


1385 1389 1390 1406 1407 1410 
   1    1    3    1    2    1 

In [44]:
exc = meta[meta$SubStudy %in% 'Exercise', ]
dim(exc)

[1] 324   8

In [45]:
head(table(exc$CollectionDate))


1063 1064 1071 1072 1083 1084 
   8    1    8    1    8    1 

### clinical tests

In [46]:
tests = read.csv('../application_2_T2D/data/clinical_tests.txt', sep='\t')

In [47]:
dim(tests)

[1] 969  57

In [48]:
head(tests)

VisitID,A1C,AG,ALB,ALCRU,ALKP,ALT,AST,BASO,BASOAB,...,TGL,TP,UALB,UALBCR,WBC,SubjectID,CL1,CL2,CL3,CL4
ZOZOW1T-1013,6.0,8,4.0,,96,48,22,0.6,0.04,...,43,6.3,,,6.0,69-001,D7,,Infection_Late,Infection
ZOZOW1T-1015,5.9,8,4.2,,103,77,120,0.9,0.04,...,75,6.5,,,5.0,69-001,D30,,Infection_Recovery_Late,Infection_L
ZOZOW1T-1021,6.3,,NA,173.5,NA,,,1.0,0.09,...,46,NA,7,<30,8.9,69-001,D1,,Infection_Early,Infection
ZOZOW1T-1022,6.1,7,4.2,278.2,69,40,27,0.5,0.05,...,41,6.6,16,<30,10.8,69-001,D3,,Infection_Middle,Infection
ZOZOW1T-1023,6.3,13,4.2,412.8,66,53,31,0.6,0.04,...,57,6.7,18,<30,7.0,69-001,D15,,Infection_Recovery_Early,Infection_L
ZOZOW1T-1025,6.4,10,4.0,299.7,76,67,31,1.0,0.05,...,41,6.5,14,<30,5.0,69-001,D35,,Infection_Recovery_Late,Infection_L


In [50]:
length(intersect(meta$VisitID, tests$VisitID))

[1] 954

### Multi-omics

In [12]:
microb = read.csv('../application_2_T2D/data/gut_16s_abundance.txt', sep='\t')

In [13]:
dim(microb)

[1] 855  97

In [14]:
head(microb)

SampleID,phylum_Actinobacteria,phylum_Bacteroidetes,phylum_Firmicutes,phylum_Proteobacteria,phylum_Verrucomicrobia,phylum_unclassified_Bacteria,class_Actinobacteria,class_Bacilli,class_Bacteroidia,...,genus_Veillonella,genus_unclassified_Bacteria,genus_unclassified_Clostridiales,genus_unclassified_Clostridiales_Incertae.Sedis.XIII,genus_unclassified_Coriobacteriaceae,genus_unclassified_Erysipelotrichaceae,genus_unclassified_Firmicutes,genus_unclassified_Lachnospiraceae,genus_unclassified_Porphyromonadaceae,genus_unclassified_Ruminococcaceae
ZOZOW1T-1010,0.000449469,0.6508661,0.2250804,0.007364381,0.015835148,0.09743111,0.000449469,0.000138298,0.6508661,...,0.000000000,0.09743111,0.007883000,0.000034600,0.000414895,0.002385645,0.015696850,0.02745220,0.000000000,0.07412786
ZOZOW1T-1011,0.000175673,0.7305179,0.1848078,0.003899937,0.008186354,0.07090155,0.000175673,0.000070300,0.7305179,...,0.000035100,0.07090155,0.008713372,0.000140538,0.000175673,0.000808095,0.008607969,0.01180521,0.000000000,0.06921509
ZOZOW1T-1012,0.000597467,0.1783637,0.7968613,0.000358480,0.002788178,0.02091134,0.000597467,0.000119493,0.1783637,...,0.000079700,0.02091134,0.022902892,0.000517805,0.000398311,0.001712738,0.000876285,0.01836214,0.000000000,0.61535091
ZOZOW1T-1015,0.000057500,0.7407258,0.2265165,0.005976840,0.001206862,0.02497055,0.000057500,0.000028700,0.7407258,...,0.000028700,0.02497055,0.013045602,0.000086200,0.000028700,0.004109077,0.007729663,0.04384931,0.000028700,0.06709577
ZOZOW1T-1021,0.001112673,0.5545795,0.3641954,0.038357929,0.000058600,0.04128602,0.001112673,0.000058600,0.5545795,...,0.000409932,0.04128602,0.006207543,0.000117123,0.000175685,0.000468494,0.000995549,0.13258374,0.015753104,0.04462403
ZOZOW1T-1022,0.001603395,0.3950483,0.4880453,0.038009903,0.000282952,0.07493516,0.001603395,0.000047200,0.3950012,...,0.000235793,0.07493516,0.022258901,0.000000000,0.000424428,0.001886348,0.010280594,0.10818203,0.005376091,0.06889885


In [15]:
metab = read.csv('../data/metabolome_abundance.txt', sep='\t')

In [16]:
dim(metab)

[1] 981 730

In [17]:
head(metab)

SampleID,pHILIC_142.1225_1.3,pHILIC_199.0825_1.5,pHILIC_154.1225_1.6,pHILIC_302.3046_1.8,pHILIC_790.5707_2,pHILIC_257.1127_2,pHILIC_114.0661_2.5,pHILIC_776.5554_3,pHILIC_137.0457_3.1,...,nRPLC_206.0821_3.4,nRPLC_188.0352_3.1,nRPLC_151.04_2.3,nRPLC_203.002_1.9,nRPLC_117.0557_0.6,SubjectID,CL1,CL2,CL3,CL4
ZOZOW1T-01,48933.50,12930.38,8806.556,161984.40,7.346514e+00,116880.70,36174800,527681.50,2602094,...,935.6534,3259.925,1333.717,1804.745,85970.57,69-001,D0,UNK_I,Infection_Early,Infection
ZOZOW1T-02,38733.88,59952.15,10619.350,216496.40,2.236876e+03,312380.90,78940410,1179171.00,4149377,...,4870.6410,16948.620,96603.690,234939.000,456850.90,69-001,D4,UNK_II,Infection_Middle,Infection
ZOZOW1T-03,42435.48,38987.30,7809.121,108672.90,2.856187e+04,104521.10,39027150,336941.30,3691144,...,1984.9850,4993.880,5273.242,63674.750,163617.20,69-001,D21,UNK_III,Infection_Recovery_Early,Infection_L
ZOZOW1T-05,46083.25,15084.49,8909.069,97748.90,1.508881e+01,139771.50,50265620,159105.70,3473635,...,4684.4880,5450.713,5575.404,52257.440,115033.30,69-001,,UNK_V,,Healthy
ZOZOW1T-06,52522.11,31970.83,9438.909,56578.23,8.179492e-01,95810.41,33657510,88254.79,2381462,...,3527.4080,2736.235,3822.664,39924.610,100389.00,69-001,,UNK_VI,,Healthy
ZOZOW1T-07,55143.94,66744.44,10622.140,241043.10,9.357697e+00,310446.10,93627340,1860479.00,11346390,...,11543.1500,22860.450,10171.170,102154.400,270623.50,69-001,,UNK_VII,,Healthy


In [18]:
protein = read.csv('../application_2_T2D/data/proteome_abundance.txt', sep='\t')

In [19]:
dim(protein)

[1] 950 308

In [20]:
head(protein)

SampleID,IGLL5,MASP2,APOL1,CEP290,CD5L,FCN3,ATRN,ATRN.1,APOM,...,AFG3L2,LYVE1,FCGBP,ZNF10,FAM161B,SubjectID,CL1,CL2,CL3,CL4
ZOZOW1T-1013,2.8890910,-3.485240,2.132138,-4.976074,-1.26468800,1.0038730,-1.1664690,-2.992944,2.661346,...,-4.092226,-3.221125,-1.842757,-4.067586,-4.967957,69-001,D7,,Infection_Late,Infection
ZOZOW1T-1015,3.5575810,-5.187031,2.344245,-3.159375,-1.78812200,0.4237069,-0.3813586,-2.145862,2.875468,...,-2.354904,-4.209470,-3.957724,-5.360170,-5.433125,69-001,D30,,Infection_Recovery_Late,Infection_L
ZOZOW1T-1021,3.0075250,-3.538002,2.075924,-5.796386,-0.09918693,0.7123992,-1.0418710,-2.023487,2.810211,...,-4.657650,-2.812472,-1.663895,-1.682703,-5.757788,69-001,D1,,Infection_Early,Infection
ZOZOW1T-1022,0.8637556,-3.620376,1.965186,-6.757326,1.68225500,0.4037232,-1.3362490,-1.117450,3.030691,...,-4.457742,-3.100214,-1.893479,-4.537362,-6.926365,69-001,D3,,Infection_Middle,Infection
ZOZOW1T-1023,0.8247913,-3.374524,1.733176,-4.988127,-1.22475600,0.6819563,-1.0456650,-1.378891,1.851585,...,-4.352034,-3.225586,-1.946335,-4.634648,-5.068154,69-001,D15,,Infection_Recovery_Early,Infection_L
ZOZOW1T-1025,2.4006020,-3.472215,2.317851,-5.464828,-0.40461060,0.5579242,-1.2727810,-2.085027,2.644164,...,-4.439872,-3.158289,-1.297884,-4.164654,-5.598330,69-001,D35,,Infection_Recovery_Late,Infection_L


In [21]:
cytok = read.csv('../application_2_T2D/data/cytokine_abundance.txt', sep='\t')

In [22]:
dim(cytok)

[1] 967  67

In [23]:
head(cytok)

SampleID,IL17F,FASL,TGFA,MIP1A,SDF1A,IL27,LIF,IL1B,IL2,...,PDGFBB,VEGF,LEPTIN,PAI1,CD40L,ENA78,CHEX1,CHEX2,CHEX3,CHEX4
ZOZOW1T-1012,184.4445,13.60762,20.58748,57.73917,180.0586,22.06917,39.09397,21.49778,36.66082,...,110.62097,328.4199,24.90527,9902.316,412.6311,141.0739,10469.596,858.3563,1461.456,12.78310
ZOZOW1T-1013,147.3604,19.31404,17.33683,52.92757,158.5744,23.90827,41.09879,21.49778,30.37611,...,141.09818,282.1784,24.90527,8968.832,175.1344,165.7225,9957.126,798.5442,1312.449,12.78310
ZOZOW1T-1015,138.5773,14.04658,14.08617,53.46219,174.4318,22.98872,37.59035,24.30184,27.75748,...,110.62097,296.3662,25.42413,8677.616,345.0719,152.0871,10165.621,751.8488,1260.149,12.78310
ZOZOW1T-1021,155.6555,17.55822,17.33683,53.46219,164.7127,24.36804,34.08192,19.62841,30.37611,...,88.04526,325.7926,35.28247,8934.923,313.8907,126.9141,10167.570,704.6287,1198.473,12.29145
ZOZOW1T-1022,176.6373,18.43613,26.00524,59.87766,154.4821,26.66691,37.08915,20.56310,26.18630,...,99.89751,362.5756,42.54651,8892.537,392.8831,104.8877,10042.375,825.8269,1348.961,14.25808
ZOZOW1T-1023,182.4927,17.55822,17.33683,63.08539,152.4360,23.90827,38.09156,21.49778,35.61337,...,119.65126,364.1520,26.98071,8914.478,243.2133,145.7939,10012.660,692.0366,1163.935,13.76642


### merge data

In [60]:
idx_selected = Reduce(intersect, list(meta$VisitID, tests$VisitID, microb$SampleID, 
                                      metab$SampleID, protein$SampleID, cytok$SampleID))
length(idx_selected)

[1] 651

In [87]:
meta_sub = meta[meta$VisitID %in% idx_selected, ]
tests_sub = tests[tests$VisitID%in% idx_selected,]

In [88]:
dim(meta_sub); dim(tests_sub)

[1] 651   8

[1] 651  57

In [92]:
table(meta_sub$SubStudy)


Exercise      HMP 
       0      651 

In [61]:
microb_sub = microb[microb$SampleID %in% idx_selected, ]
metab_sub = metab[metab$SampleID %in% idx_selected, ]
protein_sub = protein[protein$SampleID %in% idx_selected, ]
cytok_sub = cytok[cytok$SampleID %in% idx_selected, ]

In [62]:
dim(microb_sub); dim(metab_sub); dim(protein_sub); dim(cytok_sub)

[1] 651  97

[1] 651 730

[1] 651 308

[1] 652  67

In [66]:
cytok_sub$SampleID[duplicated(cytok_sub$SampleID)]

[1] ZLGD9M0-05
960 Levels: ZIS22OE-02 ZJOSZHK-01 ZJOSZHK-02 ZJTKAE3-02 ... ZYXQKWY-09

In [67]:
cytok_sub[cytok_sub$SampleID %in% 'ZLGD9M0-05', ]

,SampleID,IL17F,FASL,TGFA,MIP1A,SDF1A,IL27,LIF,IL1B,IL2,...,PDGFBB,VEGF,LEPTIN,PAI1,CD40L,ENA78,CHEX1,CHEX2,CHEX3,CHEX4
827,ZLGD9M0-05,846.0738,183.6236,114.6922,174.0188,626.5699,163.0845,474.1457,95.73158,142.3667,...,197.6034,1247.25,813.293,8832.77,4248.709,719.1029,10666.13,862.1522,1547.361,18.53955
828,ZLGD9M0-05,846.0738,183.6236,114.6922,174.0188,626.5699,163.0845,474.1457,95.73158,142.3667,...,197.6034,1247.25,813.293,8832.77,4248.709,719.1029,10666.13,862.1522,1547.361,18.53955


In [69]:
# remove duplicated sampleID in cytok
cytok_sub = distinct(cytok_sub, SampleID, .keep_all = TRUE)
dim(cytok_sub)

[1] 651  67

## compute shannon diversity for each omics data

In [73]:
microb_matrix = microb_sub %>% remove_rownames %>% column_to_rownames(var="SampleID")
microb_sub$shannon_microb  = diversity(microb_matrix, index = "shannon") 
dim(microb_sub)
summary(microb_sub$shannon_microb)

[1] 651  98

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.101   2.383   2.473   2.449   2.543   2.747 

In [80]:
metab_matrix = metab_sub %>% remove_rownames %>% 
                    select (-c(SubjectID, CL1, CL2, CL3, CL4)) %>% 
                    column_to_rownames(var="SampleID")
metab_sub$shannon_metab  = diversity(metab_matrix, index = "shannon") 
dim(metab_sub)
summary(metab_sub$shannon_metab)

[1] 651 731

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.09974 4.47522 4.56768 4.35115 4.62369 4.85247 

In [85]:
protein_matrix = protein_sub %>% remove_rownames %>% 
                    select (-c(SubjectID, CL1, CL2, CL3, CL4)) %>%  column_to_rownames(var="SampleID")
protein_sub$shannon_protein  = diversity(exp(protein_matrix), index = "shannon") 
dim(protein_sub)
summary(protein_sub$shannon_protein)

[1] 651 309

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.8989  1.6362  1.9211  1.8851  2.1456  2.6035 

In [86]:
cytok_matrix = cytok_sub %>% remove_rownames %>% column_to_rownames(var="SampleID")
cytok_sub$shannon_cytok  = diversity(cytok_matrix, index = "shannon") 
dim(cytok_sub)
summary(cytok_sub$shannon_cytok)

[1] 651  68

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  2.026   2.283   2.362   2.378   2.447   2.994 

 ### export data

In [102]:
df = merge(meta_sub, tests_sub, by='VisitID')
colnames(df)[1] = 'SampleID'
df = merge(df, microb_sub[, c('SampleID', 'shannon_microb')], by='SampleID')
df = merge(df, metab_sub[, c('SampleID', 'shannon_metab')], by='SampleID')
df = merge(df, protein_sub[, c('SampleID', 'shannon_protein')], by='SampleID')
df = merge(df, cytok_sub[, c('SampleID', 'shannon_cytok')], by='SampleID')
dim(df)

[1] 651  68

In [103]:
head(df)

SampleID,SubjectID.x,CollectionDate,Event,Event_Note1,Event_Note2,Event_Note3,SubStudy,A1C,AG,...,WBC,SubjectID.y,CL1,CL2,CL3,CL4,shannon_microb,shannon_metab,shannon_protein,shannon_cytok
ZIS22OE-02,ZIS22OE,132,Healthy,,,,HMP,5.7,7,...,5.8,69-006,,,,Healthy,2.513490,4.513828,1.507388,2.406467
ZJOSZHK-01,ZJOSZHK,1224,Healthy,,,,HMP,5.1,9,...,5.6,69-125,,,,Healthy,2.384329,4.424113,1.657185,2.132517
ZJTKAE3-02,ZJTKAE3,328,Weight-gain,,,,HMP,5.5,6,...,5.9,70-1015,,,,Weight-gain,2.579071,4.601622,1.646427,2.325557
ZJTKAE3-04,ZJTKAE3,488,Healthy,,,,HMP,5.2,8,...,5.2,70-1015,,,,Healthy,2.594636,4.683781,1.730919,2.306839
ZJTKAE3-06,ZJTKAE3,721,Healthy,,,,HMP,5.5,12,...,5.1,70-1015,,,,Healthy,2.478193,1.013396,1.630764,2.331847
ZJTKAE3-2012,ZJTKAE3,581,Imz,D1,flu?,Imz_Early,HMP,5.1,5,...,4.9,70-1015,D1,flu?,Imz_Early,Imz,2.325915,4.623535,2.209811,2.351109


In [153]:
#write.table(df, file='../application_2_T2D/data/HMP2_t2d.txt', sep='\t', row.names=F)

### Visulization

In [133]:
fig_microb <- ggplot(df, aes(x=CollectionDate, y=shannon_microb)) + geom_line(alpha=0.5) + guides(colour=FALSE) + 
                xlab("CollectionDate") + ylab("Microbial Shannon") + aes(colour = factor(SubjectID.x)) +
                theme_classic() + theme(axis.text = element_text(color='black')) +
                theme(axis.title.x = element_text(face="bold"),
                      axis.title.y = element_text(face="bold")) +
                scale_fill_distiller(palette = "Spectral") 

fig_metab <- ggplot(df, aes(x=CollectionDate, y=shannon_metab)) + geom_line(alpha=0.5) + guides(colour=FALSE) + 
                xlab("CollectionDate") + ylab("Molecular Shannon") + aes(colour = factor(SubjectID.x)) +
                theme_classic() + theme(axis.text = element_text(color='black')) +
                theme(axis.title.x = element_text(face="bold"),
                      axis.title.y = element_text(face="bold")) +
                scale_fill_distiller(palette = "Spectral")

fig_protein <- ggplot(df, aes(x=CollectionDate, y=shannon_protein)) + geom_line(alpha=0.5) + guides(colour=FALSE) + 
                xlab("CollectionDate") + ylab("Protein Shannon") + aes(colour = factor(SubjectID.x)) +
                theme_classic() + theme(axis.text = element_text(color='black')) +
                theme(axis.title.x = element_text(face="bold"),
                      axis.title.y = element_text(face="bold")) +
                scale_fill_distiller(palette = "Spectral")

fig_cytok <- ggplot(df, aes(x=CollectionDate, y=shannon_cytok)) + geom_line(alpha=0.5) + guides(colour=FALSE) + 
                xlab("CollectionDate") + ylab("Cytok Shannon") + aes(colour = factor(SubjectID.x)) +
                theme_classic() + theme(axis.text = element_text(color='black')) +
                theme(axis.title.x = element_text(face="bold"),
                      axis.title.y = element_text(face="bold")) +
                scale_fill_distiller(palette = "Spectral")

In [158]:
# png('../figures/visu_omics.png', units="in", width=10, height=7, res=300)
# grid.arrange(fig_microb, fig_metab, fig_protein, fig_cytok, nrow = 2)
# dev.off()

pdf 
  2

In [151]:
# HSCRP is flat
fig_A1C <- ggplot(df, aes(x=CollectionDate, y=A1C)) + geom_line(alpha=0.5) + guides(colour=FALSE) + 
                xlab("CollectionDate") + ylab("A1C") + aes(colour = factor(SubjectID.x)) +
                theme_classic() + theme(axis.text = element_text(color='black')) +
                theme(axis.title.x = element_text(face="bold"),
                      axis.title.y = element_text(face="bold")) +
                scale_fill_distiller(palette = "Spectral")
                
fig_GLU <- ggplot(df, aes(x=CollectionDate, y=GLU)) + geom_line(alpha=0.5) + guides(colour=FALSE) + 
                xlab("CollectionDate") + ylab("GLU") + aes(colour = factor(SubjectID.x)) +
                theme_classic() + theme(axis.text = element_text(color='black')) +
                theme(axis.title.x = element_text(face="bold"),
                      axis.title.y = element_text(face="bold")) +
                scale_fill_distiller(palette = "Spectral")       

fig_NEUT <- ggplot(df, aes(x=CollectionDate, y=NEUT)) + geom_line(alpha=0.5) + guides(colour=FALSE) + 
                xlab("CollectionDate") + ylab("NEUT") + aes(colour = factor(SubjectID.x)) +
                theme_classic() + theme(axis.text = element_text(color='black')) +
                theme(axis.title.x = element_text(face="bold"),
                      axis.title.y = element_text(face="bold")) +
                scale_fill_distiller(palette = "Spectral") 

fig_LDL <- ggplot(df, aes(x=CollectionDate, y=LDL)) + geom_line(alpha=0.5) + guides(colour=FALSE) + 
                xlab("CollectionDate") + ylab("LDL") + aes(colour = factor(SubjectID.x)) +
                theme_classic() + theme(axis.text = element_text(color='black')) +
                theme(axis.title.x = element_text(face="bold"),
                      axis.title.y = element_text(face="bold")) +
                scale_fill_distiller(palette = "Spectral") 

In [159]:
# png('../figures/visu_tests.png', units="in", width=10, height=7, res=300)
# grid.arrange(fig_A1C, fig_GLU, fig_NEUT, fig_LDL, nrow = 2)
# dev.off()

Warning message:
“Removed 1 rows containing missing values (geom_path).”Warning message:
“Removed 1 rows containing missing values (geom_path).”Warning message:
“Removed 3 rows containing missing values (geom_path).”Warning message:
“Removed 2 rows containing missing values (geom_path).”

pdf 
  2